In [1]:
!pip install -U datasets
!pip install -U accelerate
!pip install -U transformers
!pip install -U huggingface_hub

  Using cached pyarrow-17.0.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (3.3 kB)
  Using cached pandas-2.2.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached frozenlist-1.4.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached charset_normalizer-3.3.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (33 kB)
  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 8.6 MB/s eta 0:00:00
Using cached fsspec-2024.6.1-py3-none-any.whl (177 kB

In [2]:
from datasets import load_dataset

dataset = load_dataset("ilsilfverskiold/tech-keywords-topics-summary")
dataset

/opt/miniconda3/envs/plp_module/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 635/635 [00:00<00:00, 350214.73 examples/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'source', 'text', 'timestamp', 'reactions', 'engagement', 'url', 'text_length', 'keywords', 'topic', 'summary', '__index_level_0__'],
        num_rows: 7196
    })
    validation: Dataset({
        features: ['id', 'source', 'text', 'timestamp', 'reactions', 'engagement', 'url', 'text_length', 'keywords', 'topic', 'summary', '__index_level_0__'],
        num_rows: 635
    })
    test: Dataset({
        features: ['id', 'source', 'text', 'timestamp', 'reactions', 'engagement', 'url', 'text_length', 'keywords', 'topic', 'summary', '__index_level_0__'],
        num_rows: 635
    })
})

In [3]:
def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Text: {example['text']}'")
        print(f"'>> Keywords: {example['keywords']}'")


show_samples(dataset)


'>> Text: Driverless car users will not be prosecuted for fatal crashes in UK'
'>> Keywords: Driverless Cars, Legal Issues, UK'

'>> Text: Google is embedding inaudible watermarks right into its AI generated music -'
'>> Keywords: Google, AI Music, Watermarks, Audio Technology'

'>> Text: What are your thoughts on Nextjs performance? Do you agree with this chart? - ( by 10up where Nextjs appears lower than WordPress on core vitals. Couldn’t post the image here due to community rules. But appreciate any other studies and thought you have on this matter.'
'>> Keywords: Next.js, Performance, 10up, WordPress'


In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = 'facebook/bart-large' # go smaller if you can
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

texts = dataset['train']['text']

# Tokenize all texts and find the maximum length (max for BART is 1024 tokens)
max_token_length = max(len(tokenizer.encode(text, truncation=True)) for text in texts)
print(f"The longest text is {max_token_length} tokens long.")

/opt/miniconda3/envs/plp_module/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


The longest text is 245 tokens long.


In [5]:
def get_feature(batch):
  encodings = tokenizer(batch['text'], text_target=batch['keywords'],
                        max_length=1024, truncation=True)

  encodings = {'input_ids': encodings['input_ids'],
               'attention_mask': encodings['attention_mask'],
               'labels': encodings['labels']}

  return encodings

dataset_pt = dataset.map(get_feature, batched=True)
dataset_pt

Map: 100%|██████████| 635/635 [00:00<00:00, 22430.94 examples/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'source', 'text', 'timestamp', 'reactions', 'engagement', 'url', 'text_length', 'keywords', 'topic', 'summary', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 7196
    })
    validation: Dataset({
        features: ['id', 'source', 'text', 'timestamp', 'reactions', 'engagement', 'url', 'text_length', 'keywords', 'topic', 'summary', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 635
    })
    test: Dataset({
        features: ['id', 'source', 'text', 'timestamp', 'reactions', 'engagement', 'url', 'text_length', 'keywords', 'topic', 'summary', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 635
    })
})

In [6]:
columns = ['input_ids', 'labels', 'attention_mask']
dataset_pt.set_format(type='torch', columns=columns)

In [7]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir = 'bart_tech_keywords', # rename to what you want it to be called
    num_train_epochs=3, # your choice
    warmup_steps = 500,
    per_device_train_batch_size=4, 
    per_device_eval_batch_size=4,
    weight_decay = 0.01,
    logging_steps = 10,
    evaluation_strategy = 'steps',
    eval_steps=50, 
    save_steps=1e6,
    gradient_accumulation_steps=16 
)

trainer = Trainer(model=model, args=training_args, tokenizer=tokenizer, data_collator=data_collator,
                  train_dataset = dataset_pt['train'], eval_dataset = dataset_pt['validation'])

trainer.train()

/opt/miniconda3/envs/plp_module/lib/python3.12/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/336 [00:00<?, ?it/s]/opt/miniconda3/envs/plp_module/lib/python3.12/site-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
  3%|▎         | 10/336 [01:20<37:17,  6.86s/it] 

{'loss': 3.6513, 'grad_norm': 40.85301971435547, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.09}


  6%|▌         | 20/336 [02:41<45:16,  8.60s/it]

{'loss': 3.1626, 'grad_norm': 31.181283950805664, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.18}


  9%|▉         | 30/336 [03:56<35:56,  7.05s/it]

...

 98%|█████████▊| 330/336 [2:13:27<01:21, 13.51s/it]

{'loss': 0.8061, 'grad_norm': 10.157153129577637, 'learning_rate': 3.3e-05, 'epoch': 2.93}


100%|██████████| 336/336 [2:14:41<00:00, 12.53s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
100%|██████████| 336/336 [2:14:45<00:00, 24.07s/it]

{'train_runtime': 8085.979, 'train_samples_per_second': 2.67, 'train_steps_per_second': 0.042, 'train_loss': 1.1810229207788194, 'epoch': 2.99}


TrainOutput(global_step=336, training_loss=1.1810229207788194, metrics={'train_runtime': 8085.979, 'train_samples_per_second': 2.67, 'train_steps_per_second': 0.042, 'total_flos': 3582300094562304.0, 'train_loss': 1.1810229207788194, 'epoch': 2.9883268482490273})

In [16]:
trainer.save_model('tech-keywords-extractor_plp')


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [31]:
# !huggingface-cli login --token hf_xxxxxxxxxxxx

# check output of fine tuned model
from transformers import pipeline

pipe = pipeline('summarization', model='tech-keywords-extractor_plp')

text_test = "Apple is launching iphone 16 with Apple Intelligence. It also uses OpenAI ChatGPT"

# test_text=dataset['test'][0]['text']
# keywords = dataset['test'][0]['keywords']
print("the text: ", text_test)
print("generated keywords: ", pipe(text_test))
# print("orginal keywords : ",keywords)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 128, but your input_length is only 21. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)


the text:  Apple is launching iphone 16 with Apple Intelligence. It also uses OpenAI ChatGPT
generated keywords:  [{'summary_text': 'Apple, iphone 16, Apple Intelligence, OpenAI, ChatGPT'}]


In [29]:
# Check back on output of base untrained model

pipe = pipeline('summarization', model = 'facebook/bart-large')

text_test = "Apple is launching iphone 16 with Apple Intelligence. It also uses OpenAI ChatGPT. "

# test_text=dataset['test'][0]['text']
# keywords = dataset['test'][0]['keywords']
print("the text: ", text_test)
print("generated keywords: ", pipe(text_test))
# print("orginal keywords : ",keywords)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 128, but your input_length is only 23. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)


the text:  Apple is launching iphone 16 with Apple Intelligence. It also uses OpenAI ChatGPT. 
generated keywords:  [{'summary_text': 'Apple is launching iphone 16 with Apple Intelligence. It also uses OpenAI ChatGPT. '}]


In [30]:
# Check back on output of base model fine tuned on CNN daily news

pipe = pipeline('summarization', model = 'facebook/bart-large-cnn')

text_test = "Apple is launching iphone 16 with Apple Intelligence. It also uses OpenAI ChatGPT. "

# test_text=dataset['test'][0]['text']
# keywords = dataset['test'][0]['keywords']
print("the text: ", text_test)
print("generated keywords: ", pipe(text_test))
# print("orginal keywords : ",keywords)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 142, but your input_length is only 23. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)


the text:  Apple is launching iphone 16 with Apple Intelligence. It also uses OpenAI ChatGPT. 
generated keywords:  [{'summary_text': "Apple is launching iphone 16 with Apple Intelligence. It also uses OpenAI ChatGPT. Apple is launching Apple Intelligence with Apple Artificial Intelligence on the iPhone 16. Apple  also using OpenAI chatGPT on the phone. Apple's new iPhone 16 will be released on September 18."}]


In [20]:
for i in range(0, 50):
    text_test = dataset['test'][i]['text']
    keywords = dataset['test'][i]['keywords']
    print("text: ", text_test)
    print("generated keywords: ", pipe(text_test)[0]['summary_text'])
    print("original keywords: ", keywords)

Your max_length is set to 128, but your input_length is only 74. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=37)


text:  Unleash the Edge: Women’s Faux Leather Zip-Up Motorcycle Jacket — Vintage Grunge Aesthetic Revived - Ride the Rebel Wave: Women’s Faux Leather Zip-Up Motorcycle Jacket — A Fusion of Timeless Edge and Vintage Grunge Aesthetic for…


Your max_length is set to 128, but your input_length is only 12. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)


generated keywords:  Women's Faux Leather Zip-Up Motorcycle Jacket, Vintage Grunge Aesthetic
original keywords:  Women's Fashion, Faux Leather, Motorcycle Jacket, Vintage Grunge
text:  Distractions, analytical thinking and falling for fake news


Your max_length is set to 128, but your input_length is only 20. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)


...

generated keywords:  LLMs, Kernels, Research Roundup
original keywords:  LLMs, Kernels, Research
text:  Alan Wake 2 is an unexpected visual marvel even on older GPUsxfire.com
generated keywords:  Alan Wake 2, GPUs, Visual Wonders
original keywords:  Alan Wake 2, GPUs, Visual Marvel
text:  I am worried with how AI is improving propaganda - If you’ve been following the war in the Middle East on Twitter, you’ve probably noticed how difficult it’s been to distinguish what is real and what isn’t. Without getting into the gruesome details, a few days ago there was an image going around of a deceased child and there was heavy discussion into whether it was ai generated or real. There are some ai identifying tools but they appear to be very unreliable at the moment, so we don’t have a concrete way to make this distinction. I don’t know what the final findings were for that particular image, but either way it exemplifies how AI can be used to make dangerous propaganda. Apparently Google is deve

Your max_length is set to 128, but your input_length is only 30. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)


generated keywords:  AI, Propaganda, Google, Digital Watermark, Legislation
original keywords:  AI, Propaganda, Digital Watermark, Google, Legislation
text:  Frank & FuelStat Launch - This month the Fuelet Wallet team has introduced two analytic tools for the Fuel Network users. Those are:


Your max_length is set to 128, but your input_length is only 26. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)


generated keywords:  Frank, FuelStat, Fuelet Wallet, Fuel Network
original keywords:  Frank, FuelStat, Fuelet Wallet, Fuel Network, Analytics Tools
text:  Big Tech on Defensive as Copyright Threat Looms Over Generative AI. - Photo by Xu Haiwei on Unsplash
generated keywords:  Big Tech, Copyright Threat, Generative AI, Xu Haiwei
original keywords:  Big Tech, Copyright, Generative AI, Xu Haiwei


In [24]:
# you would replace your own name here
# you do not need to create a repository beforehand
trainer.push_to_hub("wbcmthh42/tech-keywords-extractor_plp")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

training_args.bin: 100%|██████████| 5.18k/5.18k [00:00<00:00, 18.3kB/s]






































































































































































































































































































































































































































CommitInfo(commit_url='https://huggingface.co/wbcmthh42/bart_tech_keywords/commit/aaa476e530d52c5ef3c58dd4029c1cdf966f6d6c', commit_message='wbcmthh42/tech-keywords-extractor_plp', commit_description='', oid='aaa476e530d52c5ef3c58dd4029c1cdf966f6d6c', pr_url=None, pr_revision=None, pr_num=None)